# Testing C++ code in python
June 12, 2020

In [1]:
import numpy as np
from scipy.fftpack import fft, fft2, ifft2, fftshift


Steps to implement spectrum calculation
- Compute FFT
- Shift frequencies
- Compute radial average

## Read input in C++

In [15]:
# ### Save single 2D image to csv for C++ code to handle
# fname='data/gen_imgs.npy'
# fname='data/images.csv'
# a1=np.loadtxt(fname)
# print(a1.shape)
# # np.savetxt('data/images.csv',a1[0,:5,:5],delimiter=',',newline=',')

## FFT

#### 1D FFT

In [31]:
y1=[(1.0*10.0*(e+1))-(1.0j*(e+1)) for e in range(6)]
y1=[i+1 for i in range(6)]
print(y1)

[1, 2, 3, 4, 5, 6]


In [34]:
z1=fft(y1)
z1=fftshift(fft(y1))

for i in z1:
    print(i)

(-3+0j)
(-2.999999999999999-1.7320508075688776j)
(-3.000000000000001-5.196152422706632j)
(21+0j)
(-3.000000000000001+5.196152422706632j)
(-2.999999999999999+1.7320508075688776j)


### 2D FFT

In [ ]:
y2=np.array([(10.0*(x+1)+5*y)+(1.0j*(((x+3)+y*2.0))) for y in range(5) for x in range(5)]).reshape(5,5)
y2


In [ ]:
z2=fft2(y2)

In [ ]:
for i in z2: 
    for j in i:
        print(np.around(j,decimals=4),end='\t')
    print('')
#     print([np.around(j,decimals=4) for j in i])

## Frequency shift

In [ ]:
! cat data/images.csv

In [ ]:
freqs = np.fft.fftfreq(9, d=1./9)
print(freqs)
print(np.fft.fftshift(freqs))

freqs_2d=freqs.reshape(3,3)
print(freqs_2d)
np.fft.fftshift(freqs_2d, axes=(1,))

In [ ]:
a1=np.arange(-2,10)
print(a1)
np.fft.fftshift(a1)

In [ ]:
### C++ like code
for i in range(3):
    for j in range(3):
        print(freqs_2d[i,j],'\t',end='')
    print()
        

## Radial profile

In [ ]:
## Optimized python code
def f_radial_profile(data, center=(None,None)):
    ''' Module to compute radial profile of a 2D image '''
    y, x = np.indices((data.shape)) # Get a grid of x and y values
    
    if center[0]==None and center[1]==None:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0]) # compute centers
        
    # get radial values of every pair of points
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    
    # Compute histogram of r values
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel()) 
    radialprofile = tbin / nr
    
    return radialprofile

In [ ]:
x_size,y_size=4,4
x_max,x_min=x_size-1,0
y_max,y_min=y_size-1,0

center_x=(x_max-x_min)/2.0
center_y=(y_max-y_min)/2.0
print(center_x,center_y)

In [ ]:
img_arr=np.random.randint(0,10,size=x_size*y_size).reshape(x_size,y_size)
img_arr

In [ ]:
img_arr=np.array([23, 6, 17, 35,33, 15, 26, 12,9, 21, 2, 27,10,19,3,6]).reshape(4,4)
print(img_arr)

In [ ]:
max_r=np.sqrt((x_size-center_x)**2+(y_size-center_y)**2).astype(int)
r_bins=np.zeros(max_r,dtype=int)
r_arr=np.zeros(max_r,dtype=np.float64)
r_prof=np.zeros(max_r,dtype=np.float64)

for x in range(x_min,x_size):
    for y in range(y_min,y_size):
        r=np.sqrt((x-center_x)**2+(y-center_y)**2)
        r_int=np.int(r)
        r_bins[r_int]+=1
        r_arr[r_int]+=img_arr[x,y]
        print(x,y,r,r_int,img_arr[x,y])

### Take ratios to get the radial profile
for i in np.arange(len(r_arr)):
    if r_bins[i]!=0: 
        r_prof[i]=(r_arr[i]/r_bins[i])



In [ ]:
r_bins,r_arr,r_prof

In [ ]:
f_radial_profile(img_arr)